In [14]:
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, Date, ForeignKey, BigInteger
from sqlalchemy_utils import database_exists, create_database
from config import user, password, host, database
import pandas as pd
import numpy as np

In [17]:
from sqlalchemy import create_engine
db_string = f"mysql+pymysql://{user}:{password}@{host}:3306/{database}"
engine = create_engine(db_string)
Base = declarative_base()

Database model using SQLAlchemy

In [21]:
class Continent(Base):
    __tablename__ = 'continents'
    id_continent = Column(Integer, primary_key=True)
    continent = Column(String(50))

class Country(Base):
    __tablename__ = 'countries'
    id_country = Column(Integer, primary_key=True)
    country = Column(String(50))
    id_continent = Column(Integer, ForeignKey('continents.id_continent'))

class Population(Base):
    __tablename__ = 'live_population'
    id_data = Column(Integer, primary_key=True)
    measurement = Column(BigInteger)
    date_measurement = Column(Date)
    id_country = Column(Integer, ForeignKey('countries.id_country'))

class Year(Base):
    __tablename__ = 'years'
    id_year = Column(Integer, primary_key=True)
    year = Column(Integer)

class Age(Base):
    __tablename__ = 'ages'
    id_age = Column(Integer, primary_key=True)
    age = Column(Integer)

class Death(Base):
    __tablename__ = 'deaths'
    id_data = Column(Integer, primary_key=True)
    measurement = Column(BigInteger)
    sex = Column(String(10))
    id_year = Column(Integer, ForeignKey('years.id_year'))
    id_country = Column(Integer, ForeignKey('countries.id_country'))
    id_age = Column(Integer, ForeignKey('ages.id_age'))

class Birth(Base):
    __tablename__ = 'births'
    id_data = Column(Integer, primary_key=True)
    measurement = Column(BigInteger)
    id_year = Column(Integer, ForeignKey('years.id_year'))
    id_country = Column(Integer, ForeignKey('countries.id_country'))
    id_mother_age = Column(Integer, ForeignKey('ages.id_age'))

class Marriage(Base):
    __tablename__ = 'marriages'
    id_data = Column(Integer, primary_key=True)
    measurement = Column(BigInteger)
    id_year = Column(Integer, ForeignKey('years.id_year'))
    id_age = Column(Integer, ForeignKey('ages.id_age'))
    id_country = Column(Integer, ForeignKey('countries.id_country'))

Connection to the database

In [5]:
if not database_exists(engine.url):
    create_database(engine.url)
else:
    engine.connect()
    Base.metadata.create_all(engine)

Load data from CSV into DataFrame


In [6]:
birth_death_df = pd.read_csv('data-finally/birth_death.csv')
married_df = pd.read_csv('data-finally/married.csv')
country_to_continent_df = pd.read_csv('data-finally/countryContinent.csv', encoding = 'ISO-8859-1')

Add the `kontynent` column to the data in married_df and birth_death_df


In [34]:
for df in [married_df, birth_death_df]:
    df["kontynent"] = [country_to_continent_df[country_to_continent_df["country"] == country]["continent"].values[0] if country in country_to_continent_df["country"].unique() else None for country in df["kraj"]]

Add data to the continents table


In [35]:
continents = pd.DataFrame(married_df["kontynent"].unique(), columns=['continent'])
continents.index.name = 'id_continent'
continents.to_sql('continents', engine, if_exists = "append")

In [66]:
continents

,continent
id_continent,
0,Asia
1,Americas
2,Africa
3,Europe


Add data to the countries table

In [36]:
countries = pd.DataFrame(married_df["kraj"].unique(), columns=['country'])
countries.index.name = 'id_country'
countries["id_continent"] = [continents[continents["continent"] == married_df[married_df["kraj"] == country]["kontynent"].values[0]].index.values[0] for country in married_df["kraj"].unique()]
countries.to_sql('countries', engine, if_exists = "append")

In [63]:
countries

,country,id_continent
id_country,,
0,Bangladesh,0
1,Brazil,1
2,China,0
3,Dem. Rep. of the Congo,2
4,Ethiopia,2
5,Germany,3
6,India,0
7,Indonesia,0
8,Iran (Islamic Republic of),0


Add data to the years table


In [37]:
years_birth_death = pd.DataFrame(married_df["rok"].unique(), columns=['year'])
years_married = pd.DataFrame(birth_death_df["rok"].unique(), columns=['year'])
years = pd.concat([years_birth_death, years_married]).drop_duplicates().reset_index(drop=True)
years.index.name = 'id_year'
years.to_sql('years', engine, if_exists = "append")

In [23]:
years.head()

,year
id_year,
0,1970
1,1971
2,1972
3,1973
4,1974


Add data to the ages table


In [38]:
age_married = pd.DataFrame(married_df["wiek"].unique() , columns=['age']).astype(str)
ages_birth_death  = pd.DataFrame(birth_death_df["wiek"].unique() , columns=['age']).astype(str)
ages = pd.concat([age_married, ages_birth_death]).drop_duplicates().reset_index(drop=True)
ages.index.name = 'id_age'
ages.to_sql('ages', engine, if_exists = "append")

In [20]:
ages.head()

,age
id_age,
0,15
1,16
2,17
3,18
4,19


Add data to the births table


In [39]:
births_df = birth_death_df[birth_death_df["Indykator"] == "Births by age of mother (1-year)"]
births_df.head()

,kraj,rok,płeć,wiek,Indykator,wartość,kontynent
0,Bangladesh,1950,Both sexes,15,Births by age of mother (1-year),49740.0,Asia
1,Bangladesh,1950,Both sexes,16,Births by age of mother (1-year),77560.0,Asia
2,Bangladesh,1950,Both sexes,17,Births by age of mother (1-year),98709.0,Asia
3,Bangladesh,1950,Both sexes,18,Births by age of mother (1-year),110663.0,Asia
4,Bangladesh,1950,Both sexes,19,Births by age of mother (1-year),112789.0,Asia


In [ ]:
new_rows = []
for _, row in births_df.iterrows():
    id_country = countries[countries["country"] == row["kraj"]].index[0]
    id_year = years[years["year"] == row["rok"]].index[0]
    id_mother_age = ages[ages["age"] == row["wiek"]].index[0]
    measurment = row["wartość"]
    new_rows.append({
        "measurement": measurment,
        "id_year": id_year,
        "id_country": id_country,
        "id_mother_age": id_mother_age
    })

births = pd.concat([pd.DataFrame(new_rows)], ignore_index=True)


In [57]:
births.head()

,measurement,id_year,id_country,id_mother_age
id_data,,,,
0,49740.0,55,0,0
1,77560.0,55,0,1
2,98709.0,55,0,2
3,110663.0,55,0,3
4,112789.0,55,0,4


In [56]:
births.index.name = 'id_data'
births.to_sql('births', engine, if_exists = "append")

52500

Add data to the deaths table


In [58]:
death_df = birth_death_df[birth_death_df["Indykator"] == "Deaths by 1-year age groups and sex"]
death_df.head()

,kraj,rok,płeć,wiek,Indykator,wartość,kontynent
52500,Bangladesh,1950,Male,0,Deaths by 1-year age groups and sex,208502.0,Asia
52501,Bangladesh,1950,Female,0,Deaths by 1-year age groups and sex,191179.0,Asia
52502,Bangladesh,1950,Both sexes,0,Deaths by 1-year age groups and sex,399681.0,Asia
52503,Bangladesh,1950,Male,1,Deaths by 1-year age groups and sex,54845.0,Asia
52504,Bangladesh,1950,Female,1,Deaths by 1-year age groups and sex,45039.0,Asia


In [63]:
new_rows_deaths = []
for _, row in death_df.iterrows():
    id_country = countries[countries["country"] == row["kraj"]].index[0]
    id_year = years[years["year"] == row["rok"]].index[0]
    id_age = ages[ages["age"] == row["wiek"]].index[0]
    measurment = row["wartość"]
    sex = row["płeć"]

    new_rows_deaths.append({
        "measurement": measurment,
        "sex": sex,
        "id_year": id_year,
        "id_country": id_country,
        "id_age": id_age
    })

deaths = pd.concat([pd.DataFrame(new_rows_deaths)], ignore_index=True)

In [65]:
deaths.head()

,measurement,sex,id_year,id_country,id_age
0,208502.0,Male,55,0,35
1,191179.0,Female,55,0,35
2,399681.0,Both sexes,55,0,35
3,54845.0,Male,55,0,36
4,45039.0,Female,55,0,36


In [66]:
deaths.index.name = 'id_data'
deaths.to_sql('deaths', engine, if_exists = "append")

454500

Add data to the marriage table


In [67]:
married_df.head()

,kraj,rok,płeć,wiek,IndicatorName,wartość,kontynent
0,Bangladesh,1970,Female,15,Currently married by age of woman (Number),446515.2,Asia
1,Bangladesh,1970,Female,16,Currently married by age of woman (Number),446515.2,Asia
2,Bangladesh,1970,Female,17,Currently married by age of woman (Number),446515.2,Asia
3,Bangladesh,1970,Female,18,Currently married by age of woman (Number),446515.2,Asia
4,Bangladesh,1970,Female,19,Currently married by age of woman (Number),446515.2,Asia


In [80]:
new_rows_marriages = []
for _, row in married_df.iterrows():
    id_country = countries[countries["country"] == row["kraj"]].index[0]
    id_year = years[years["year"] == row["rok"]].index[0]
    id_age = ages[ages["age"] == str(row["wiek"])].index[0]
    measurment = row["wartość"]

    new_rows_marriages.append({
        "measurement": measurment,
        "id_year": id_year,
        "id_age": id_age,
        "id_country": id_country

    })

marriages = pd.concat([pd.DataFrame(new_rows_marriages)], ignore_index=True)

In [81]:
marriages.head()

,measurement,id_year,id_age,id_country
0,446515.2,0,0,0
1,446515.2,0,1,0
2,446515.2,0,2,0
3,446515.2,0,3,0
4,446515.2,0,4,0


In [84]:
marriages.index.name = 'id_data'
marriages.to_sql('marriages', engine, if_exists = "append")

38500

# Population live- web scraping


In [ ]:
from sqlalchemy.orm import sessionmaker
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

countries = [
    "India", "China", "United States of America", "Indonesia", "Pakistan", "Nigeria",
    "Brazil", "Bangladesh", "Russian Federation", "Mexico", "Ethiopia", "Japan",
    "Philippines", "Egypt", "Dem. Rep. of the Congo", "Viet Nam", "Iran (Islamic Republic of)", "Turkey",
    "Germany", "Thailand"
]

country_url_map = {
    "India": "india",
    "China": "china",
    "United States of America": "us",
    "Indonesia": "indonesia",
    "Pakistan": "pakistan",
    "Nigeria": "nigeria",
    "Brazil": "brazil",
    "Bangladesh": "bangladesh",
    "Russian Federation": "russia",
    "Mexico": "mexico",
    "Ethiopia": "ethiopia",
    "Japan": "japan",
    "Philippines": "philippines",
    "Egypt": "egypt",
    "Dem. Rep. of the Congo": "democratic-republic-of-the-congo",
    "Viet Nam": "vietnam",
    "Iran (Islamic Republic of)": "iran",
    "Turkey": "turkey",
    "Germany": "germany",
    "Thailand": "thailand"
}
Session = sessionmaker(bind=engine)
session = Session()

id_data = 0
country_id_map = {country.country: country.id_country for country in session.query(Country).all()}


chrome_options = Options()
chrome_options.add_argument("--headless")

driver = webdriver.Chrome(options=chrome_options)

def get_population(country):
    url_country = country_url_map[country]
    url = f'https://www.worldometers.info/world-population/{url_country}-population/'
    driver.get(url)
    time.sleep(2)
    wait = WebDriverWait(driver, 10)
    try:
        population_element = wait.until(EC.presence_of_element_located(
            (By.CSS_SELECTOR, '#maincounter-wrap > div > span')
        ))
        population_str = population_element.text.replace(',', '')
        population = int(population_str)
    except Exception as e:
        print(f"Could not get population data for {country}: {e}")
        population = None
    return population

try:
    while True:
        if id_data > 9999:
            id_data = 0
        for country in countries:
            population = get_population(country)
            if population:
               
                country_id = country_id_map[country]
                new_measurement = Population(
                    id_data = id_data,
                    measurement=int(population),
                    date_measurement=datetime.now().date(),
                    id_country=country_id
                )
                id_data += 1
                session.add(new_measurement)
                session.commit()
                print(f"Inserted data for {country}: {population}")
        time.sleep(300)  
except KeyboardInterrupt:
    pass
finally:
    driver.quit()
    session.close()

Inserted data for India: 1441593135
Inserted data for China: 1425183529
Inserted data for United States of America: 341796858
Inserted data for Indonesia: 279776148
Inserted data for Pakistan: 245163858
Inserted data for Nigeria: 229100089
Inserted data for Brazil: 217625560
Inserted data for Bangladesh: 174684297
Inserted data for Russian Federation: 143961766
Inserted data for Mexico: 129379447
Inserted data for Ethiopia: 129688579
Inserted data for Japan: 122637803
Inserted data for Philippines: 119089056
Inserted data for Egypt: 114467091
Inserted data for Dem. Rep. of the Congo: 105592200
Could not get population data for Viet Nam: invalid literal for int() with base 10: 'retrieving data...'
Inserted data for Iran (Islamic Republic of): 89803623
Inserted data for Turkey: 86256127
Inserted data for Germany: 83252880
Inserted data for Thailand: 71884984
Inserted data for India: 1441593287
Inserted data for China: 1425183523
Inserted data for United States of America: 341796879
Inser